In [814]:
import re, pdb

A map of all event engines.

# Classes

In [815]:
class Queue:
    def __init__(self):
        self.list = []

    def empty(self):
        return len(self.list) == 0

    def put(self, item):
        self.list.append(item)

    def get(self):
        item = self.list[0]
        self.list = self.list[1:]
        return item

In [816]:
class Event:
      def __init__(self, type='', parameters={}):
        self.type = type
        self.parameters = parameters

In [890]:
class Engine:
    def __init__(self, routines, trigger=None, items=dict()):
        self.routines = routines
        self.items = items
        
    def list_event(self, event):
        print(event.type, event.parameters)

    def list_events(self, n=10):
        for i, event in enumerate(self.queue.list[:n]):
            print("{}. ".format(i+1), event.type, event.parameters)

    def fetch_event(self, debug=False):
        event = self.queue.get()
        generated_events = self.routines[event.type](**{**self.items, **event.parameters})
        for generated_event in generated_events:
            if debug:
                self.list_event(generated_event)
            self.manage_event(generated_event)

    def fetch_exhaustive(self, debug=False):
        while not self.queue.empty():
            self.fetch_event(debug)

    def fetch_and_list(self):
        self.fetch_event()
        self.list_events()

# Other Data Structure

# Functions

## Abstraction Level 1

In [891]:
class StartingEngine(Engine):
    def __init__(self, trigger, *args):
        super().__init__(*args)
        self.queue = Queue()
        if trigger:
            self.queue.put(trigger)
        self.output = Queue()
    def manage_event(self, event):
        if event.type != 'ClassifiedChar':
            self.queue.put(event)
        else:
            self.output.put(event)

In [892]:
def get_lines(filename):
    try:
        with open(filename, 'r') as f:
            for line in f.readlines():
                yield Event("Line", {'line': line})
    except FileNotFoundError:
        return None

## Abstraction Level 2

In [893]:
def get_chars(line):
    list_lines = list(line)
    for char in list_lines:
        yield Event("Char", {'char': char})

## Abstraction Levels 3, 4 and 5

In [894]:
def classify_chars(char):
    _type = 'descartavel' if char in [' ', '\t'] else 'util' if char != '\n' else 'controle'
    if re.match(r'[a-zA-Z]', char):
        _class = 'letra'
    elif re.match(r'[0-9]', char):
        _class = 'digito'
    elif char in [' ', '\t']:
        char = 'espaco'
        _class = 'delimitador'
    elif char == '\n':
        char = '\n'
        _class = 'especial'
    else:
        _class = 'especial'
    yield Event("ClassifiedChar", {'char': char, 'char_type': _type, 'char_class': _class})

## Abstraction Level 6

In [895]:
%run 'MetaRecognizer.ipynb'

Populating the interactive namespace from numpy and matplotlib


In [896]:
mr = MetaRecognizer('grammar.txt', 'fsm/')
mr.determinizeAll()
reserved_words = mr.getSymbols()

states = {
    'A': {'next': {'letra': 'B','digito': 'C', 'especial': 'D', 'controle': 'E'}, 'final': False},
    'B': {'next': {'letra': 'B','digito': 'B'}, 'final': True, 'type':'letter'},
    'C': {'next': {'digito': 'C'}, 'final': True, 'type':'digit'},
    'D': {'next': {'especial': 'D'}, 'final': True, 'type':'especial'},
    'E': {'next': {'letra': 'E', 'digito': 'E', 'especial': 'E'}, 'final': True, 'type': 'remark'},
}

class LexicalEngine(Engine):
    def __init__(self, queue, *args):
        super().__init__(*args)
        self.queue = queue
        self.output = Queue()
        self.state = 'A'
        self.token = ''
        self.remark = False

    def manage_event(self, event):
        self.output.put(event)
                
    def lexical_analysis(self, char, char_type, char_class):
        state = self.state
        token = self.token
        if char_class in states[state]['next'].keys():
            self.state = states[state]['next'][char_class]
            self.token = token + char
        else:
            if states[state]['final']:
                if char_type in ['descartavel']:
                    self.state = 'A'
                    self.token = ''
                else:
                    self.state = states['A']['next'][char_class]
                    self.token = char
                token_type = token if token in reserved_words else states[state]['type'] 
                token_type = token if token_type == 'symbol' else token_type
                yield Event("Token", {'token': token_type, 'value': token})
            else:
                if state != 'A':
                    raise SyntaxError("Invalid token: {}".format(token))

## Abstraction Level 7

In [897]:
mr.fsms['Int'].dfa.accepting_states

['B', 'C']

In [898]:
class SyntaticalEngine(Engine):
    def __init__(self, queue, mr, *args):
        super().__init__(*args)
        self.queue = queue
        self.output = Queue()
        self.mr = mr
        self.stack = [('A', list(mr.fsms.values())[0].dfa.name, [], [])]
        
    def manage_event(self, event):
        self.output.put(event)
                
    def syntatical_analysis(self, token, value):
        state, machine, tokens, backtrackingOptions = self.stack[-1]
        paths = list(self.mr.fsms[machine].dfa.transitions[state].keys())
        nt = list(filter(lambda x: x.startswith(':'), paths))
        print("{}({})".format(token, value), state, machine, backtrackingOptions, self.mr.fsms[machine].dfa.transitions)
        print("is '{}' in {}".format(value, paths))
        if token in paths:
            print("\tNormal transition")
            self.stack[-1] = (
                self.mr.fsms[machine].dfa.transitions[state][token],
                machine,
                tokens+[{'token': token, 'value': value}],
                []
            )
        else:
            self.queue.list = [Event("Token", {'token': token, 'value': value})] + self.queue.list
            if nt:
                if not backtrackingOptions:
                    backtrackingOptions = nt
                    self.stack[-1] = (state, machine, tokens, backtrackingOptions)
                print("Trying machine "+backtrackingOptions[0])
                self.stack.append(('A', backtrackingOptions[0][1:], [], []))
            else:
                if state in mr.fsms[machine].dfa.accepting_states:
                    if len(self.stack) == 0:
                        raise SyntaxError('Pilha vazia. Erro no token {}'.format(tokens))
                    else:
                        self.stack = self.stack[:-1]
                        self.stack[-1] = tuple(list(self.stack[-1])[:-1] + [self.stack[-1][-1][1:]]) # remove bt trial
                        print("Found token '{}': {}".format(machine, tokens))
                        self.queue.list = [Event("Token", {'token': ':'+machine, 'value': tokens})] + self.queue.list#[1:]
                        yield Event("SyntaticalToken", {'type': machine, 'tokens': tokens})
                else:
                    print("Backtracking...")
                    returningTokens = [Event("Token", {'token': token, 'value': value}) for token, value in tokens]
                    self.queue.list = returningTokens + self.queue.list
                    while self.stack:
                        self.stack = self.stack[:-1]
                        backtrackingOptions = self.stack[-1][3][1:]
                        if backtrackingOptions:
                            self.stack[-1] = tuple(list(self.stack[-1])[:-1] + [backtrackingOptions])
                            break
                    else:
                        raise SyntaxError(
                            'Parou em estado de rejeição. Estados de aceitação: {}'.format(
                                mr.fsms[machine].dfa.accepting_states
                            ))

# Testing

## Instantiation

In [899]:
starting = StartingEngine(
    Event('File', {'filename': 'bubblesort.bs'}),
    {
        'File': get_lines,
        'Line': get_chars,
        'Char': classify_chars,
    }
)

In [900]:
lexical = LexicalEngine(starting.output, {})
lexical.routines = {
    'ClassifiedChar': lexical.lexical_analysis,
}

In [901]:
syntatical = SyntaticalEngine(lexical.output, mr, {})
syntatical.routines = {
    'Token': syntatical.syntatical_analysis,
}

## Running

In [902]:
starting.list_events()

1.  File {'filename': 'bubblesort.bs'}


In [903]:
starting.fetch_exhaustive(debug=True)

Line {'line': '10 LET N = 10 \n'}
Line {'line': '20 DATA 53, 77, 82, 12, 38, 90, 16, 72, 55, 62 \n'}
Line {'line': '30 DIM Z(10) \n'}
Line {'line': '35 DIM S(1000) \n'}
Line {'line': '36 LET P = 1 \n'}
Line {'line': '40 FOR I = 1 TO N \n'}
Line {'line': '50 READ Z(I) \n'}
Line {'line': '60 NEXT I \n'}
Line {'line': '1000 FOR I = 1 TO N \n'}
Line {'line': '1500 FOR J = I TO N \n'}
Line {'line': '1510 IF Z(I) <= Z(J) THEN 1800 \n'}
Line {'line': '1520 LET T = Z(J) \n'}
Line {'line': '1530 LET Z(J) = Z(I) \n'}
Line {'line': '1540 LET Z(I) = T \n'}
Line {'line': '1800 NEXT J \n'}
Line {'line': '1810 NEXT I \n'}
Line {'line': '9000 FOR I = 1 TO 10 \n'}
Line {'line': '9000 PRINT Z(I) \n'}
Line {'line': '9000 NEXT I '}
Char {'char': '1'}
Char {'char': '0'}
Char {'char': ' '}
Char {'char': 'L'}
Char {'char': 'E'}
Char {'char': 'T'}
Char {'char': ' '}
Char {'char': 'N'}
Char {'char': ' '}
Char {'char': '='}
Char {'char': ' '}
Char {'char': '1'}
Char {'char': '0'}
Char {'char': ' '}
Char {'char'

In [904]:
lexical.list_events(20000)

1.  ClassifiedChar {'char': '1', 'char_type': 'util', 'char_class': 'digito'}
2.  ClassifiedChar {'char': '0', 'char_type': 'util', 'char_class': 'digito'}
3.  ClassifiedChar {'char': 'espaco', 'char_type': 'descartavel', 'char_class': 'delimitador'}
4.  ClassifiedChar {'char': 'L', 'char_type': 'util', 'char_class': 'letra'}
5.  ClassifiedChar {'char': 'E', 'char_type': 'util', 'char_class': 'letra'}
6.  ClassifiedChar {'char': 'T', 'char_type': 'util', 'char_class': 'letra'}
7.  ClassifiedChar {'char': 'espaco', 'char_type': 'descartavel', 'char_class': 'delimitador'}
8.  ClassifiedChar {'char': 'N', 'char_type': 'util', 'char_class': 'letra'}
9.  ClassifiedChar {'char': 'espaco', 'char_type': 'descartavel', 'char_class': 'delimitador'}
10.  ClassifiedChar {'char': '=', 'char_type': 'util', 'char_class': 'especial'}
11.  ClassifiedChar {'char': 'espaco', 'char_type': 'descartavel', 'char_class': 'delimitador'}
12.  ClassifiedChar {'char': '1', 'char_type': 'util', 'char_class': 'digi

In [905]:
lexical.fetch_exhaustive(debug=True)

Token {'token': 'digit', 'value': '10'}
Token {'token': 'LET', 'value': 'LET'}
Token {'token': 'letter', 'value': 'N'}
Token {'token': '=', 'value': '='}
Token {'token': 'digit', 'value': '10'}
Token {'token': '\n', 'value': '\n'}
Token {'token': 'digit', 'value': '20'}
Token {'token': 'DATA', 'value': 'DATA'}
Token {'token': 'digit', 'value': '53'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '77'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '82'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '12'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '38'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '90'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '16'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '72'}
Token {'token': ',', 'value': ','}
Token {'token': 'digit', 'value': '55'}
Token {'token': ',', 'value': ','}
Token {'token

In [906]:
syntatical.list_events(20000)

1.  Token {'token': 'digit', 'value': '10'}
2.  Token {'token': 'LET', 'value': 'LET'}
3.  Token {'token': 'letter', 'value': 'N'}
4.  Token {'token': '=', 'value': '='}
5.  Token {'token': 'digit', 'value': '10'}
6.  Token {'token': '\n', 'value': '\n'}
7.  Token {'token': 'digit', 'value': '20'}
8.  Token {'token': 'DATA', 'value': 'DATA'}
9.  Token {'token': 'digit', 'value': '53'}
10.  Token {'token': ',', 'value': ','}
11.  Token {'token': 'digit', 'value': '77'}
12.  Token {'token': ',', 'value': ','}
13.  Token {'token': 'digit', 'value': '82'}
14.  Token {'token': ',', 'value': ','}
15.  Token {'token': 'digit', 'value': '12'}
16.  Token {'token': ',', 'value': ','}
17.  Token {'token': 'digit', 'value': '38'}
18.  Token {'token': ',', 'value': ','}
19.  Token {'token': 'digit', 'value': '90'}
20.  Token {'token': ',', 'value': ','}
21.  Token {'token': 'digit', 'value': '16'}
22.  Token {'token': ',', 'value': ','}
23.  Token {'token': 'digit', 'value': '72'}
24.  Token {'toke

In [907]:
syntatical.fetch_exhaustive()

digit(10) A Program [] {'A': {':BStatement': 'B'}, 'B': {'\n': 'C'}, 'C': {':BStatement': 'F', ':Int': 'G'}, 'F': {'\n': 'D'}, 'G': {'END': 'H'}, 'D': {'\n': 'E', ':BStatement': 'F', ':Int': 'G'}, 'E': {'\n': 'E', ':BStatement': 'F', ':Int': 'G'}, 'H': {}}
is '10' in [':BStatement']
Trying machine :BStatement
digit(10) A BStatement [] {'A': {':Int': 'B'}, 'B': {':Remark': 'K', ':Dim': 'G', ':Def': 'H', ':Read': 'L', ':Goto': 'O', ':Return': 'J', ':For': 'E', ':Data': 'M', ':Gosub': 'I', ':Print': 'N', ':Next': 'F', ':Assign': 'C', ':If': 'D'}, 'G': {}, 'K': {}, 'N': {}, 'D': {}, 'H': {}, 'J': {}, 'M': {}, 'F': {}, 'O': {}, 'I': {}, 'C': {}, 'E': {}, 'L': {}}
is '10' in [':Int']
Trying machine :Int
digit(10) A Int [] {'A': {'digit': 'B'}, 'B': {'digit': 'C'}, 'C': {'digit': 'C'}}
is '10' in ['digit']
	Normal transition
LET(LET) B Int [] {'A': {'digit': 'B'}, 'B': {'digit': 'C'}, 'C': {'digit': 'C'}}
is 'LET' in ['digit']
Found token 'Int': [{'token': 'digit', 'value': '10'}]
:Int([{'tok

) H Eb [] {'A': {':Num': 'G', 'FN': 'J', ':Predef': 'L', '(': 'B', ':Var': 'H'}, 'J': {'letter': 'K'}, 'L': {'(': 'C'}, 'B': {':Exp': 'I'}, 'K': {'(': 'C'}, 'C': {':Exp': 'D'}, 'I': {')': 'F'}, 'D': {')': 'E'}, 'F': {}, 'G': {}, 'H': {}, 'E': {}}
is '
' in []
Found token 'Eb': [{'token': ':Var', 'value': [{'token': 'letter', 'value': 'N'}]}]
:Eb([{'token': ':Var', 'value': [{'token': 'letter', 'value': 'N'}]}]) A Exp [] {'A': {'+': 'B', '-': 'C', ':Eb': 'D'}, 'B': {':Eb': 'D'}, 'C': {':Eb': 'D'}, 'D': {'-': 'G', '/': 'I', '*': 'H', '↑': 'J', '+': 'F'}, 'G': {':Eb': 'E'}, 'I': {':Eb': 'E'}, 'H': {':Eb': 'E'}, 'J': {':Eb': 'E'}, 'F': {':Eb': 'E'}, 'E': {'*': 'H', '↑': 'J', '+': 'F', '-': 'G', '/': 'I'}}
is '[{'token': ':Var', 'value': [{'token': 'letter', 'value': 'N'}]}]' in ['+', '-', ':Eb']
	Normal transition

(
) D Exp [] {'A': {'+': 'B', '-': 'C', ':Eb': 'D'}, 'B': {':Eb': 'D'}, 'C': {':Eb': 'D'}, 'D': {'-': 'G', '/': 'I', '*': 'H', '↑': 'J', '+': 'F'}, 'G': {':Eb': 'E'}, 'I': {':Eb

:Eb([{'token': ':Var', 'value': [{'token': 'letter', 'value': 'Z'}, {'token': '(', 'value': '('}, {'token': ':Exp', 'value': [{'token': ':Eb', 'value': [{'token': ':Var', 'value': [{'token': 'letter', 'value': 'J'}]}]}]}, {'token': ')', 'value': ')'}]}]) A Exp [] {'A': {'+': 'B', '-': 'C', ':Eb': 'D'}, 'B': {':Eb': 'D'}, 'C': {':Eb': 'D'}, 'D': {'-': 'G', '/': 'I', '*': 'H', '↑': 'J', '+': 'F'}, 'G': {':Eb': 'E'}, 'I': {':Eb': 'E'}, 'H': {':Eb': 'E'}, 'J': {':Eb': 'E'}, 'F': {':Eb': 'E'}, 'E': {'*': 'H', '↑': 'J', '+': 'F', '-': 'G', '/': 'I'}}
is '[{'token': ':Var', 'value': [{'token': 'letter', 'value': 'Z'}, {'token': '(', 'value': '('}, {'token': ':Exp', 'value': [{'token': ':Eb', 'value': [{'token': ':Var', 'value': [{'token': 'letter', 'value': 'J'}]}]}]}, {'token': ')', 'value': ')'}]}]' in ['+', '-', ':Eb']
	Normal transition
THEN(THEN) D Exp [] {'A': {'+': 'B', '-': 'C', ':Eb': 'D'}, 'B': {':Eb': 'D'}, 'C': {':Eb': 'D'}, 'D': {'-': 'G', '/': 'I', '*': 'H', '↑': 'J', '+': 'F'},

NEXT(NEXT) B BStatement [':Def', ':Read', ':Goto', ':Return', ':For', ':Data', ':Gosub', ':Print', ':Next', ':Assign', ':If'] {'A': {':Int': 'B'}, 'B': {':Remark': 'K', ':Dim': 'G', ':Def': 'H', ':Read': 'L', ':Goto': 'O', ':Return': 'J', ':For': 'E', ':Data': 'M', ':Gosub': 'I', ':Print': 'N', ':Next': 'F', ':Assign': 'C', ':If': 'D'}, 'G': {}, 'K': {}, 'N': {}, 'D': {}, 'H': {}, 'J': {}, 'M': {}, 'F': {}, 'O': {}, 'I': {}, 'C': {}, 'E': {}, 'L': {}}
is 'NEXT' in [':Remark', ':Dim', ':Def', ':Read', ':Goto', ':Return', ':For', ':Data', ':Gosub', ':Print', ':Next', ':Assign', ':If']
Trying machine :Def
NEXT(NEXT) A Def [] {'A': {'DEF FN': 'B'}, 'B': {'letter': 'C'}, 'C': {'(': 'D'}, 'D': {'letter': 'E'}, 'E': {')': 'G', 'digit': 'F'}, 'G': {'=': 'H'}, 'F': {')': 'G'}, 'H': {':Exp': 'I'}, 'I': {}}
is 'NEXT' in ['DEF FN']
Backtracking...
NEXT(NEXT) B BStatement [':Read', ':Goto', ':Return', ':For', ':Data', ':Gosub', ':Print', ':Next', ':Assign', ':If'] {'A': {':Int': 'B'}, 'B': {':Remar

In [811]:
syntatical.list_events(200)

In [908]:
[x.parameters for x in syntatical.output.list]

[{'type': 'Int', 'tokens': [{'token': 'digit', 'value': '10'}]},
 {'type': 'Var', 'tokens': [{'token': 'letter', 'value': 'N'}]},
 {'type': 'Int', 'tokens': [{'token': 'digit', 'value': '10'}]},
 {'type': 'Num',
  'tokens': [{'token': ':Int', 'value': [{'token': 'digit', 'value': '10'}]}]},
 {'type': 'Eb',
  'tokens': [{'token': ':Num',
    'value': [{'token': ':Int',
      'value': [{'token': 'digit', 'value': '10'}]}]}]},
 {'type': 'Exp',
  'tokens': [{'token': ':Eb',
    'value': [{'token': ':Num',
      'value': [{'token': ':Int',
        'value': [{'token': 'digit', 'value': '10'}]}]}]}]},
 {'type': 'Assign',
  'tokens': [{'token': 'LET', 'value': 'LET'},
   {'token': ':Var', 'value': [{'token': 'letter', 'value': 'N'}]},
   {'token': '=', 'value': '='},
   {'token': ':Exp',
    'value': [{'token': ':Eb',
      'value': [{'token': ':Num',
        'value': [{'token': ':Int',
          'value': [{'token': 'digit', 'value': '10'}]}]}]}]}]},
 {'type': 'BStatement',
  'tokens': [{'toke

In [813]:
syntatical.list_events()

In [694]:
'\\n' in ['\\n']

True

In [504]:
z = [1, 2, 3]
z.pop()


3

In [505]:
z

[1, 2]